In [1]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/Users/Madhukar/opt/miniconda3/envs/ee_skmr/bin/python
3.9.2 | packaged by conda-forge | (default, Feb 21 2021, 05:02:20) 
[Clang 11.0.1 ]
sys.version_info(major=3, minor=9, micro=2, releaselevel='final', serial=0)


In [2]:
import ee
import pandas as pd
import numpy as np
ee.Initialize()

In [3]:
# read in the nhd addendum file
nhd_stats = pd.read_csv("nhd_stats_AI.csv")

# read in csv file with SSURGO variables
df_m = pd.read_csv("combined_regular_clean_with_ssurgo_variables.csv")

In [4]:
nhd_stats.head()

,comid,long_comid,lat_comid,startflag,intephem,divergence,streamorde,lengthkm,gnis_name_ind,areasqkm,totdasqkm,flow_type,distup_max,distdown_max
0,179,-67.986409,46.022164,1.0,0,0.0,1.0,2.412,0,3.5550,3.5550,1,NaN,243.97200
1,181,-67.998723,46.016490,0.0,0,0.0,1.0,0.442,0,0.2898,3.8448,1,1.364,242.60800
2,183,-67.998835,46.020847,0.0,0,0.0,2.0,0.112,1,0.2043,8.1954,1,3.278,243.29601
3,185,-67.998621,46.019712,0.0,0,0.0,2.0,0.170,1,0.0369,8.2323,1,3.438,243.13600
4,843,-68.378758,46.246067,1.0,0,0.0,1.0,1.889,0,2.7486,2.7486,1,NaN,285.85901


In [24]:
# pd.read_pickle('../1. Extract NHD information from GEE/NHD_extracted_vars_2.5kmX2.5km_with_fcode_ftype/2.5kmX2.5km_nhd_variables_part1')

In [26]:
# comid's from GEE are extracted into several pickled files
# pickling is needed to be able to share easily across team members 
# join these pickled files into one full dataset

df_merged_full = []

for i in range(df_m.shape[0] // 500 + 1):
    try:
        df_temp = pd.read_pickle('../1. Extract NHD information from GEE/NHD_extracted_vars_2.5kmX2.5km_with_fcode_ftype/2.5kmX2.5km_nhd_variables_part' + str(500 * i + 1))
#         print(df_temp.columns)
        df_merged_full.append(df_temp)
    except:
        break

df_merged_full = pd.concat(df_merged_full)

In [5]:
# # convert to protocol 3 so that can read in aws
# df_merged_full = []

# for i in range(df_m.shape[0] // 500 + 1):
#     try:
#         df_temp = pd.read_pickle(('NHD_extracted_vars_2.5kmX2.5km/combined_regular_clean_with_ssurgo_nhd_variables_part' + 
#                               str(500 * i + 1)))
#         print(df_temp.shape)
#         pickle.dump(df_temp, open("NHD_extracted_vars_2.5kmX2.5km/combined_regular_clean_with_ssurgo_nhd_variables_part" + str(500 * i + 1) + "_pkl3.pkl","wb"), protocol=3)
#         df_temp2 = pd.read_pickle(('NHD_extracted_vars_2.5kmX2.5km/combined_regular_clean_with_ssurgo_nhd_variables_part' + 
#                               str(500 * i + 1)))

#     except:
#         break
#     df_merged_full.append(df_temp2)
# df_merged_full = pd.concat(df_merged_full)



In [27]:
# write this df into pickle
import pickle
pickle.dump(df_merged_full, open("2.5kmX2.5km_nhd_variables_partsmerged","wb"), protocol=3)










# Read in merged pickle file

In [29]:
df_merged_full = pd.read_pickle("2.5kmX2.5km_nhd_variables_partsmerged")

In [30]:
df_merged_full.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'jurisdiction_type', 'da_number',
       'district', 'project_name', 'longitude', 'latitude',
       'date_issued_or_denied', 'rha_determination', 'cwa_determination',
       'rha1', 'rha2', 'cwa1', 'cwa2', 'cwa3', 'cwa4', 'cwa5', 'cwa6', 'cwa7',
       'cwa8', 'cwa9', 'potential_wetland', 'index', 'Index', 'mukey',
       'hydclprs', 'aws025wta', 'drclassdcd', 'nhd_vars_wb', 'nhd_vars_fl'],
      dtype='object')

# A word on 'nhd_vars_wb' and 'nhd_vars_fl' columns (see last two columns above)

## nhd_vars_wb: this is a list of lists
- for each record, the following six features from GEE are extracted as lists and stored into a list
- [comid_list, ftype_str, gnis_id, wb_area, fl_length, fcode]
- the column is labeled nhd_vars_wb for waterbodies

In a similar fashion, there is another column for flowlines labeled nhd_vars_fl

Note: fl_length is NaN's for waterbodies and wb_area ae NaN's for flowlines

## In the following, features are extracted from the above columns and feature engineered as discussed in meetings. Pls use your judgement to help devise any new features you wish

In [31]:
# wb for waterbodies
# extract the individual lists
df_merged_full["wb_comid_list"] = df_merged_full.apply(lambda x: x.nhd_vars_wb[0], axis=1)
df_merged_full["wb_ftype_str_list"] = df_merged_full.apply(lambda x: x.nhd_vars_wb[1], axis=1)
df_merged_full["wb_gnis_id_list"] = df_merged_full.apply(lambda x: x.nhd_vars_wb[2], axis=1)
df_merged_full["wb_area_list"] = df_merged_full.apply(lambda x: x.nhd_vars_wb[3], axis=1)

# fl for flowlines
# extract the individual lists
df_merged_full["fl_comid_list"] = df_merged_full.apply(lambda x: x.nhd_vars_fl[0], axis=1)
df_merged_full["fl_ftype_str_list"] = df_merged_full.apply(lambda x: x.nhd_vars_fl[1], axis=1)
df_merged_full["fl_gnis_id_list"] = df_merged_full.apply(lambda x: x.nhd_vars_fl[2], axis=1)
df_merged_full["fl_length_list"] = df_merged_full.apply(lambda x: x.nhd_vars_fl[4], axis=1)

# Lets look at columns of interest

In [32]:
df_merged_full[df_merged_full.columns[29:39]][9:15]


,nhd_vars_wb,nhd_vars_fl,wb_comid_list,wb_ftype_str_list,wb_gnis_id_list,wb_area_list,fl_comid_list,fl_ftype_str_list,fl_gnis_id_list,fl_length_list
9,"([120052831, 22027044], [LakePond, LakePond], ...","([21980197, 21980245, 21980217, 21980207, 2198...","[120052831, 22027044]","[LakePond, LakePond]","[974076, ]","[171.202, 0.008]","[21980197, 21980245, 21980217, 21980207, 21980...","[ArtificialPath, ArtificialPath, ArtificialPat...","[, , , , , , , , 973757, , 974058]","[0.022, 1.975, 3.135, 0.725, 2.786, 0.764, 3.0..."
10,"([904140245], [LakePond], [1075813], [12045.52...","([15594559, 15594563, 15594573, 15594565, 1559...",[904140245],[LakePond],[1075813],[12045.529],"[15594559, 15594563, 15594573, 15594565, 15594...","[StreamRiver, StreamRiver, StreamRiver, Stream...","[, , 1066928, 1066928, 1066928, , , 1066928]","[0.602, 0.217, 1.254, 0.208, 7.518, 5.608, 12...."
11,"([21631197, 904140246, 21631201], [LakePond, L...","([21632385, 21632387, 21632389, 21635915, 2163...","[21631197, 904140246, 21631201]","[LakePond, LakePond, LakePond]","[967326, 970427, ]","[0.425, 6693.837, 0.946]","[21632385, 21632387, 21632389, 21635915, 21635...","[Coastline, Coastline, Coastline, Coastline, C...","[, , , , , , , ]","[2.781, 0.287, 1.895, 0.871, 10.109, 0.15, 1.1..."
12,"([15586156, 15586164], [LakePond, LakePond], [...","([15588640, 15588636, 15587674, 15587680, 1558...","[15586156, 15586164]","[LakePond, LakePond]","[1078482, 1079552]","[0.139, 0.051]","[15588640, 15588636, 15587674, 15587680, 15587...","[ArtificialPath, ArtificialPath, StreamRiver, ...","[, , , , 1066599, 1066599, 1066851]","[0.941, 0.658, 1.216, 0.505, 0.968, 3.143, 1.98]"
13,"([], [], [], [], [], [])","([15560355, 15560297, 15560169, 15560311, 1556...",[],[],[],[],"[15560355, 15560297, 15560169, 15560311, 15560...","[StreamRiver, StreamRiver, CanalDitch, CanalDi...","[968024, 968024, , , , , , , ]","[0.766, 4.526, 1.44, 1.941, 0.95, 0.641, 0.372..."
14,"([], [], [], [], [], [])","([15594581, 15594569, 15594591], [StreamRiver,...",[],[],[],[],"[15594581, 15594569, 15594591]","[StreamRiver, StreamRiver, StreamRiver]","[, 1044708, 1044708]","[1.943, 2.539, 4.12]"


## Lets look at one row in individual columns

In [33]:
# looking at waterbodies list
df_merged_full.nhd_vars_wb[9]
# you can see there are 6 items in the list [comid_list, ftype_str, gnis_id, wb_area, fl_length, fcode]
# note that fcode is going to be null due to coding lapse

([120052831, 22027044],
 ['LakePond', 'LakePond'],
 ['974076', ''],
 [171.202, 0.008],
 [nan, nan],
 [39004, 39004])

In [34]:
# looking at flowlines list
df_merged_full.nhd_vars_fl[9]

([21980197,
  21980245,
  21980217,
  21980207,
  21980203,
  21980199,
  21980195,
  21980193,
  21978319,
  21978365,
  21978323],
 ['ArtificialPath',
  'ArtificialPath',
  'ArtificialPath',
  'ArtificialPath',
  'ArtificialPath',
  'ArtificialPath',
  'ArtificialPath',
  'ArtificialPath',
  'StreamRiver',
  'StreamRiver',
  'StreamRiver'],
 ['', '', '', '', '', '', '', '', '973757', '', '974058'],
 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
 [0.022, 1.975, 3.135, 0.725, 2.786, 0.764, 3.016, 1.652, 3.419, 3.557, 2.136],
 [55800, 55800, 55800, 55800, 55800, 55800, 55800, 55800, 46006, 46006, 46006])

In [35]:
# list of comids in waterbodies
df_merged_full.wb_comid_list[9]

[120052831, 22027044]

In [36]:
# list of comids in flowlines
df_merged_full.fl_comid_list[9]

[21980197,
 21980245,
 21980217,
 21980207,
 21980203,
 21980199,
 21980195,
 21980193,
 21978319,
 21978365,
 21978323]

### .... and so on

In [37]:
# ### Filter out invalid comids (although not used in this notebook)
# # "invalid" = present in GEE but not present in nhd_stats

# df_merged["wb_comid_list_filtered"] = df_merged.apply(lambda x: [comid for comid in x.nhd_vars_wb[0] if comid in np.array(nhd_stats.comid)
#                                                                 ], axis=1)

# df_merged["fl_comid_list_filtered"] = df_merged.apply(lambda x: [comid for comid in x.nhd_vars_fl[0] if comid in np.array(nhd_stats.comid)
#                                                                 ], axis=1)

In [38]:
# Assigned begin and end of records for each person
# MADHUKAR: records 1 - 5000
# SHOBHA: records 5000 - 10000
# RADHIKA: records 10000 - 15000

In [39]:
# Extract features present in nhd_stats for corresponding comid
# read in fl_comid_list, pull out matching variable values in nhd_stats

df_merged = df_merged_full

def extract_feature(comid, feature):
    """
    Extract features present in nhd_stats for corresponding comid
    """
    if comid == None:
        return np.nan # if no comid's in GEE
    extracted_feature = nhd_stats[nhd_stats["comid"] == comid][str(feature)]
    try:
        extracted_feature = np.array(extracted_feature).item() 
    except Exception as e:
        return np.nan # if comid in GEE but not in nhd database
    return extracted_feature


def extract_sum(feature):
    """
    feature engineering per excel sheet
    """
    return (df_merged.apply(lambda x: np.sum(np.array([extract_feature(comid, str(feature))
                                                                 for comid in x.fl_comid_list])
                                                       [~np.isnan(np.array([extract_feature(comid, str(feature))
                                                                            for comid in x.fl_comid_list]))]), 
                                                axis=1))
def extract_count(feature):
    """
    feature engineering per excel sheet
    """
    return (df_merged.apply(lambda x: len(np.array([extract_feature(comid, str(feature))
                                                                 for comid in x.fl_comid_list])
                                                       [~np.isnan(np.array([extract_feature(comid, str(feature))
                                                                            for comid in x.fl_comid_list]))]), 
                                                  axis=1))


In [40]:
# flowline variables

# areasqkm
df_merged["fl_areasqkm_sum"] = extract_sum("areasqkm")
df_merged["fl_areasqkm_count"] = extract_count("areasqkm")
df_merged["fl_areasqkm_mean"] = (df_merged.apply(lambda x: 
                                                 (x.fl_areasqkm_sum/x.fl_areasqkm_count) 
                                                 if x.fl_areasqkm_count != 0 
#                                                  else np.nan, axis=1)) # here you want to return 0
                                                 else 0, axis=1))
# gnis_name_ind
df_merged["fl_gnis_name_ind_sum"] = extract_sum("gnis_name_ind")
df_merged["fl_gnis_name_ind_count"] = extract_count("gnis_name_ind")
df_merged["fl_gnis_name_ind_mean"] = (df_merged.apply(lambda x: 
                                                 (x.fl_gnis_name_ind_sum/x.fl_gnis_name_ind_count) 
                                                 if x.fl_gnis_name_ind_count != 0 
#                                                  else np.nan, axis=1))
                                                 else 0, axis=1))

# totdasqkm
df_merged["fl_totdasqkm_sum"] = extract_sum("totdasqkm")
df_merged["fl_totdasqkm_count"] = extract_count("totdasqkm")
df_merged["fl_totdasqkm_mean"] = (df_merged.apply(lambda x: 
                                                 (x.fl_totdasqkm_sum/x.fl_totdasqkm_count) 
                                                 if x.fl_totdasqkm_count != 0 
#                                                  else np.nan, axis=1))
                                                 else 0, axis=1))

# flow_type
df_merged["fl_flow_type_sum"] = extract_sum("flow_type")
df_merged["fl_flow_type_count"] = extract_count("flow_type")
df_merged["fl_flow_type_mean"] = (df_merged.apply(lambda x: 
                                                 (x.fl_flow_type_sum/x.fl_flow_type_count) 
                                                 if x.fl_flow_type_count != 0 
#                                                  else np.nan, axis=1))
                                                 else 0, axis=1))

                                                  
# streamorde
df_merged["fl_streamorde_sum"] = extract_sum("streamorde")
df_merged["fl_streamorde_count"] = extract_count("streamorde")
df_merged["fl_streamorde_mean"] = (df_merged.apply(lambda x: 
                                                 (x.fl_streamorde_sum/x.fl_streamorde_count) 
                                                 if x.fl_streamorde_count != 0 
#                                                  else np.nan, axis=1))
                                                 else 0, axis=1))                                                   

# intephem
df_merged["fl_intephem_sum"] = extract_sum("intephem")
df_merged["fl_intephem_count"] = extract_count("intephem")
df_merged["fl_intephem_mean"] = (df_merged.apply(lambda x: 
                                                 (x.fl_intephem_sum/x.fl_intephem_count) 
                                                 if x.fl_intephem_count != 0 
#                                                  else np.nan, axis=1))
                                                 else 0, axis=1))                                                 

# startflag
df_merged["fl_startflag_sum"] = extract_sum("startflag")
df_merged["fl_startflag_count"] = extract_count("startflag")
df_merged["fl_startflag_mean"] = (df_merged.apply(lambda x: 
                                                 (x.fl_startflag_sum/x.fl_startflag_count) 
                                                 if x.fl_startflag_count != 0 
#                                                  else np.nan, axis=1))
                                                 else 0, axis=1))                                                  

# divergence
df_merged["fl_divergence_sum"] = extract_sum("divergence")
df_merged["fl_divergence_count"] = extract_count("divergence")
df_merged["fl_divergence_mean"] = (df_merged.apply(lambda x: 
                                                 (x.fl_divergence_sum/x.fl_divergence_count) 
                                                 if x.fl_divergence_count != 0 
#                                                  else np.nan, axis=1))
                                                 else 0, axis=1))                                                   

# In a similar fasion, you can feature engineer the waterbodies (I will get that later today/tomorrow and push the updated dataframe)

In [41]:
df_merged.describe()

,Unnamed: 0,Unnamed: 0.1,longitude,latitude,rha_determination,cwa_determination,rha1,rha2,cwa1,cwa2,...,fl_streamorde_mean,fl_intephem_sum,fl_intephem_count,fl_intephem_mean,fl_startflag_sum,fl_startflag_count,fl_startflag_mean,fl_divergence_sum,fl_divergence_count,fl_divergence_mean
count,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,...,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000
mean,7309.000000,7309.000000,-90.491852,37.209405,0.104932,0.369588,0.070935,0.062727,0.093235,0.063889,...,1.615928,1.297490,5.167453,0.271995,1.410425,4.350708,0.337638,0.369861,4.350708,0.037571
std,4220.286128,4220.286128,15.418083,6.766997,0.306476,0.482710,0.256725,0.242479,0.290771,0.244564,...,1.455323,1.905433,6.459320,0.349860,1.494997,4.537728,0.348352,1.817414,4.537728,0.147239
min,0.000000,0.000000,-174.197880,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3654.500000,3654.500000,-93.638850,32.815575,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,7309.000000,7309.000000,-88.186500,35.478300,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.250000,0.000000,4.000000,0.000000,1.000000,3.000000,0.250000,0.000000,3.000000,0.000000
75%,10963.500000,10963.500000,-80.226955,41.403750,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,2.000000,7.000000,0.500000,2.000000,6.000000,0.500000,0.000000,6.000000,0.000000
max,14618.000000,14618.000000,144.828420,70.483790,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,10.000000,30.000000,254.000000,1.000000,10.000000,58.000000,1.000000,57.000000,58.000000,2.000000


In [42]:
# These are the variables that needs to be made accessible to manipulation via csv files
# wb_ftype_str_list
# wb_area_list
# fl_ftype_str_list
# fl_length_list


In [43]:
df = df_merged

# wb for waterbodies
# extract the individual lists
df["wb_comid_str"] = df.apply(lambda x: "+".join([str(comid) for comid in x.nhd_vars_wb[0]]), axis=1)
df["wb_ftype_str"] = df.apply(lambda x: "+".join([str(ftype) for ftype in x.nhd_vars_wb[1] if ftype != None]), axis=1)
df["wb_gnis_id_str"] = df.apply(lambda x: "+".join([str(gnis) for gnis in x.nhd_vars_wb[2]]), axis=1)

# # sum and mean
df["wb_area_sum"] = df.apply(lambda x: np.sum(np.array([area for area in x.nhd_vars_wb[3] if area != None])), axis=1)
df["wb_area_count"] = df.apply(lambda x: len([area for area in x.nhd_vars_wb[3] if area != None]), axis=1)
# df["wb_area_mean"] = df.apply(lambda x: (x.wb_area_sum / x.wb_area_count) if x.wb_area_count != 0 else np.nan, axis=1)
df["wb_area_mean"] = df.apply(lambda x: (x.wb_area_sum / x.wb_area_count) if x.wb_area_count != 0 else 0, axis=1)

df["wb_gnis_name_ind_sum"] = df.apply(lambda x: np.sum(np.array([int(gnis) for gnis in x.nhd_vars_wb[2] if gnis not in ["", None]])), axis=1)
df["wb_gnis_name_ind_count"] = df.apply(lambda x: len([gnis for gnis in x.nhd_vars_wb[2] if gnis not in ["", None]]), axis=1)
# df["wb_gnis_name_ind_mean"] = df.apply(lambda x: (x.wb_gnis_name_ind_sum / x.wb_gnis_name_ind_count) if x.wb_gnis_name_ind_count != 0 else np.nan, axis=1)
df["wb_gnis_name_ind_mean"] = df.apply(lambda x: (x.wb_gnis_name_ind_sum / x.wb_gnis_name_ind_count) if x.wb_gnis_name_ind_count != 0 else 0, axis=1)


# # fl for flowlines
# # extract the individual lists
df["fl_comid_str"] = df.apply(lambda x: "+".join([str(comid) for comid in x.nhd_vars_fl[0]]), axis=1)
df["fl_ftype_str"] = df.apply(lambda x: "+".join([str(ftype) for ftype in x.nhd_vars_fl[1]]), axis=1)
df["fl_gnis_id_str"] = df.apply(lambda x: "+".join(x.nhd_vars_fl[2]), axis=1)

# # sum and mean
df["fl_length_sum"] = df.apply(lambda x: np.sum(np.array([length for length in x.nhd_vars_wb[4] if length != None])), axis=1)
df["fl_length_count"] = df.apply(lambda x: len([length for length in x.nhd_vars_wb[4] if length != None]), axis=1)
# df["fl_length_mean"] = df.apply(lambda x: (x.fl_length_sum / x.fl_length_count) if x.fl_length_count != 0 else np.nan, axis=1)
df["fl_length_mean"] = df.apply(lambda x: (x.fl_length_sum / x.fl_length_count) if x.fl_length_count != 0 else 0, axis=1)


In [44]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'jurisdiction_type', 'da_number',
       'district', 'project_name', 'longitude', 'latitude',
       'date_issued_or_denied', 'rha_determination', 'cwa_determination',
       'rha1', 'rha2', 'cwa1', 'cwa2', 'cwa3', 'cwa4', 'cwa5', 'cwa6', 'cwa7',
       'cwa8', 'cwa9', 'potential_wetland', 'index', 'Index', 'mukey',
       'hydclprs', 'aws025wta', 'drclassdcd', 'nhd_vars_wb', 'nhd_vars_fl',
       'wb_comid_list', 'wb_ftype_str_list', 'wb_gnis_id_list', 'wb_area_list',
       'fl_comid_list', 'fl_ftype_str_list', 'fl_gnis_id_list',
       'fl_length_list', 'fl_areasqkm_sum', 'fl_areasqkm_count',
       'fl_areasqkm_mean', 'fl_gnis_name_ind_sum', 'fl_gnis_name_ind_count',
       'fl_gnis_name_ind_mean', 'fl_totdasqkm_sum', 'fl_totdasqkm_count',
       'fl_totdasqkm_mean', 'fl_flow_type_sum', 'fl_flow_type_count',
       'fl_flow_type_mean', 'fl_streamorde_sum', 'fl_streamorde_count',
       'fl_streamorde_mean', 'fl_intephem_sum', 'fl_intephem_count'

In [45]:
import pickle

# use protocol 3 for backwards compatibility with Python 3.6 on AWS
pickle.dump(df, open("2.5kmX2.5km_nhd_variables_extracted","wb"), protocol=3)
df.to_csv("2.5kmX2.5km_nhd_variables_extracted.csv")

In [46]:
# df_merged.to_pickle("combined_regular_with_ssurgo_nhd_2.5kmX2.5km")

In [47]:
# df.to_csv("combined_regular_with_ssurgo_nhd_2.5kmX2.5km.csv")
# df.to_pickle("combined_regular_with_ssurgo_nhd_2.5kmX2.5km")

In [48]:
# df.to_csv("combined_regular_with_ssurgo_nhd_200mX200m.csv")
# df.to_pickle("combined_regular_with_ssurgo_nhd_200mX200m")

In [49]:
# with open("combined_regular_with_ssurgo_nhd_2.5kmX2.5km_pkl3.pkl", "rb") as f:
#     df_readfrompkl = pickle.load(f)

df = pd.read_pickle("2.5kmX2.5km_nhd_variables_extracted")
df.describe()

,Unnamed: 0,Unnamed: 0.1,longitude,latitude,rha_determination,cwa_determination,rha1,rha2,cwa1,cwa2,...,fl_divergence_mean,wb_area_sum,wb_area_count,wb_area_mean,wb_gnis_name_ind_sum,wb_gnis_name_ind_count,wb_gnis_name_ind_mean,fl_length_sum,fl_length_count,fl_length_mean
count,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,...,14619.000000,14619.000000,14619.000000,14619.000000,1.461900e+04,14619.000000,1.461900e+04,6908.0,14619.000000,6908.0
mean,7309.000000,7309.000000,-90.491852,37.209405,0.104932,0.369588,0.070935,0.062727,0.093235,0.063889,...,0.037571,239.810967,0.865107,174.044895,1.890082e+05,0.196525,1.495429e+05,0.0,1.343320,0.0
std,4220.286128,4220.286128,15.418083,6.766997,0.306476,0.482710,0.256725,0.242479,0.290771,0.244564,...,0.147239,3184.600853,1.846926,2542.119172,5.657580e+05,0.537191,3.998410e+05,0.0,2.038685,0.0
min,0.000000,0.000000,-174.197880,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,0.000000,0.0
25%,3654.500000,3654.500000,-93.638850,32.815575,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,0.000000,0.0
50%,7309.000000,7309.000000,-88.186500,35.478300,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,1.000000,0.0
75%,10963.500000,10963.500000,-80.226955,41.403750,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.051000,1.000000,0.030875,0.000000e+00,0.000000,0.000000e+00,0.0,2.000000,0.0
max,14618.000000,14618.000000,144.828420,70.483790,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,57535.316000,26.000000,57516.647000,1.148393e+07,7.000000,2.036537e+06,0.0,26.000000,0.0


In [50]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'jurisdiction_type', 'da_number',
       'district', 'project_name', 'longitude', 'latitude',
       'date_issued_or_denied', 'rha_determination', 'cwa_determination',
       'rha1', 'rha2', 'cwa1', 'cwa2', 'cwa3', 'cwa4', 'cwa5', 'cwa6', 'cwa7',
       'cwa8', 'cwa9', 'potential_wetland', 'index', 'Index', 'mukey',
       'hydclprs', 'aws025wta', 'drclassdcd', 'nhd_vars_wb', 'nhd_vars_fl',
       'wb_comid_list', 'wb_ftype_str_list', 'wb_gnis_id_list', 'wb_area_list',
       'fl_comid_list', 'fl_ftype_str_list', 'fl_gnis_id_list',
       'fl_length_list', 'fl_areasqkm_sum', 'fl_areasqkm_count',
       'fl_areasqkm_mean', 'fl_gnis_name_ind_sum', 'fl_gnis_name_ind_count',
       'fl_gnis_name_ind_mean', 'fl_totdasqkm_sum', 'fl_totdasqkm_count',
       'fl_totdasqkm_mean', 'fl_flow_type_sum', 'fl_flow_type_count',
       'fl_flow_type_mean', 'fl_streamorde_sum', 'fl_streamorde_count',
       'fl_streamorde_mean', 'fl_intephem_sum', 'fl_intephem_count'